## winpython内部でyolov8_ocd仮想環境を作成（コマンドプロンプト使用）\
python -m venv [仮想環境名　今回はyolov5_ocd]\
yolov8_ocdフォルダが作成されるので\(Scriptsフォルダに作成されるかも)
cd yolov8_ocdで移動して\
Scripts\activate　で仮想環境を起動するとコマンドプロンプトがこの様になるはず\
(yolov8_ocd) D:\WPy64-31090\yolov8_ocd>\　みたいになればOK
codeでVScodeを起動　カーネルが仮想環境カーネルとなっている事を確認\

## 初期設定

In [1]:
#VS codeの中で現在のディレクトリを確認
import os
path = os.getcwd()
print(path)

d:\WPy64-31090\scripts\yolov8_ocd\ultralytics


#yolov8フォルダに移動

In [2]:
os.chdir('../')

In [3]:
path = os.getcwd()
print(path)

d:\WPy64-31090\scripts\yolov8_ocd


In [ ]:
#ultralyticsのフォルダをクローンしてくる
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

In [ ]:
#必要なライブラリをインストールする
!pip install -r requirements.txt

In [7]:
#pysimpleGUIをインストール
!pip install pysimplegui

  Using cached PySimpleGUI-4.60.4-py3-none-any.whl (509 kB)



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


yolov8の動作確認
https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg
ジダンの画像をダウンロードしてカレントディレクトリに保存

In [4]:
os.chdir("D:/WPy64-31090/scripts/yolov8_ocd/ultralytics")
%pwd

'D:\\WPy64-31090\\scripts\\yolov8_ocd\\ultralytics'

In [5]:
#試験検出　学習済みモデルはyolov8xとする　最初はダウンロードに時間がかかる
from ultralytics import YOLO
model = YOLO("yolov8x.pt") 
results = model('zidane.jpg',save=False,save_txt=False,max_det=1)

d:\WPy64-31090\scripts\yolov8_ocd\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ultralytics YOLOv8.0.46  Python-3.10.9 torch-1.13.1+cpu CPU
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

image 1/1 D:\WPy64-31090\scripts\yolov8_ocd\ultralytics\zidane.jpg: 384x640 1 person, 1580.1ms
Speed: 2.0ms preprocess, 1580.1ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
#webcameraからの推論
#高速化のためにyoov8nを使用
#ocdを検出するときは訓練済みのptモデルをyolov8/ultralyticsフォルダにおいてmodel=''で名前を指定する
from ultralytics import YOLO
import torch
import cv2
from ultralytics.yolo.data.augment import LetterBox
from ultralytics.yolo.utils.plotting import Annotator, colors
from ultralytics.yolo.utils import ops
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

model = YOLO("yolov8n.pt")
#ocd検出モデルを読み込む場合
#model = YOLO("tolov8n_ocd")
cap = cv2.VideoCapture(0)


def preprocess(img, size=640):
        img = LetterBox(size, True)(image=img)
        img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        img = np.ascontiguousarray(img)  # contiguous
        img = torch.from_numpy(img)
        img = img.float()  # uint8 to fp16/32
        img /= 255  # 0 - 255 to 0.0 - 1.0
        return img.unsqueeze(0)

def postprocess(preds, img, orig_img):
    preds = ops.non_max_suppression(preds,
                                    0.25,
                                    0.8,
                                    agnostic=False,
                                    max_det=100)

    for i, pred in enumerate(preds):
        shape = orig_img.shape
        pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], shape).round()

    return preds

def drow_bbox(pred, names, annotator):
    for *xyxy, conf, cls in reversed(pred):
        c = int(cls)  # integer class
        label =  f'{names[c]} {conf:.2f}'
        annotator.box_label(xyxy, label, color=colors(c, True))


while True:
    ret, img = cap.read()
    origin = deepcopy(img)
    #imageを小さくする
    origin = cv2.resize(origin, (640,480))
    annotator = Annotator(origin,line_width=3,example=str(model.model.names))
    img = preprocess(img)
    preds = model.model(img, augment=False)
    preds = postprocess(preds,img,origin)
    drow_bbox(preds[0], model.model.names, annotator)
    cv2.imshow("test",origin)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break  # q キーを押したら終了する。

cap.release()
cv2.destroyAllWindows()


##pysimple GUI   をつかってGUI化

In [7]:
from tkinter import *

root = Tk()

monitor_height = root.winfo_screenheight()
monitor_width = root.winfo_screenwidth()
  
print("width x height = %d x %d (pixels)" %(monitor_width, monitor_height))


width x height = 1500 x 1000 (pixels)


In [9]:
#cocomodelを読み込む場合
model = YOLO("yolov8n.pt")
#ocd検出モデルを読み込む場合
#model = YOLO("yolov8n_ocd.pt")

#torchhubモデルの検出時のパラメーター設定
"""model.conf=0.4
model.iou=0.45
model.multi_label=False
model.max_det=5"""

import PySimpleGUI as sg
#すべての警告の非表示
import warnings
warnings.simplefilter('ignore')

#value = [0, 1, 2]#cameraの番号設定
threshould=0.4 #閾値の初期設定
#カメラ番号入力のポップアップ 外部入力なら1か2
cameranum = sg.popup_get_text("input camera number(0,1,2)", title="camera number", default_text="1")
cameranum =int(cameranum)

#GUIの初期設定
sg.theme('DarkBlue15')
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
   #[[sg.Text('select camera'),sg.Listbox(value, size=(10, 3),key=('cameranum'))]],
   [[sg.Text('confidence thredshold'),sg.Slider(range=(0.1,1.0),default_value=0.4, resolution=0.1 ,orientation='h',
   size=(20,2),enable_events=True, key='slider',)]],
    [[sg.Button('Start', size=(10, 1)), sg.Button('Stop', size=(10,1))]],

    [sg.Button('Exit', size=(10, 1))],]
#記録イベントのためにstartedという関数を定義               
started= False
#webカメラで検出
import cv2
import time
import datetime as dt


#camera = cv2.VideoCapture('./data/images/ocd.mp4')
#pysimpleGUIのwindow定義
window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(monitor_width,monitor_height),resizable=True,finalize=True)
event, values = window.read(timeout=20)
#cameranum = int(values['cameranum'])
#if event == ['caneranum']:
#    cameranum=int(values['cameranum'])
cap = cv2.VideoCapture(cameranum) #外部入力カメラを使用する場合0を1に変更
# ビデオ記録用の変数定義
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('result_{}.mp4'.format(dt.datetime.now()), fourcc, fps, (width, height))
while True:
    ret, imgs = cap.read()
    if not ret:
        break
    #imageを小さくする
    imgs = cv2.resize(imgs, (int(monitor_width/2),int(monitor_height/2)))
    h,w,ch=imgs.shape
# キャプチャーした画像をpngに変換
    image1 = cv2.imencode('.png', imgs)[1].tobytes()                   
# Imageの内容を更新
    window['img1'].update(data=image1)#web cameraの画像を直接表示
    event, values = window.read(timeout=0)
    """if event == ['caneranum']:
        cameranum=int(values['cameranum'])
        cap=cv2.VideoCapture(cameranum)
        ret,img = cap.read()"""

    if values['slider']:
      threshould=values['slider']
      model.conf=threshould

    if event == 'Start':
        started = True
    if started == True:
        #画面を10*10分割して1/10のところから9/10のところまで切り抜き
        im_cropped=imgs[round(h/10):round(h/10)*9, round(w/10):round(w/10)*9,:]
        #切り抜いた画像をモニターのサイズに合わせて拡大
        im_cropped=cv2.resize(im_cropped,(int(monitor_width/2),int(monitor_height/2)))
        #切り抜いた画像に対して推論
        origin = deepcopy(im_cropped)
        annotator = Annotator(origin,line_width=3,example=str(model.model.names))
        img = preprocess(im_cropped)
        preds = model.model(img, augment=False)
        preds = postprocess(preds,img,origin)
        drow_bbox(preds[0], model.model.names, annotator)       
    # キャプチャーした画像をpngに変換
        image2 = cv2.imencode('.png', origin)[1].tobytes()
    # Imageの内容を更新
        window['img2'].update(data=image2)       

    if event =='Stop':
        started = False
        out.release()

    if event == 'Exit' or event == sg.WIN_CLOSED:
        break

cap.release()    
window.close()

: 